In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import altair as alt
from datetime import datetime
import warnings
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import altair as alt

import nedu_prediction as nedu

# show all columns in the dataframe
pd.set_option('max_columns', None)

warnings.filterwarnings("ignore")

# show all columns in the dataframe
pd.set_option('max_columns', None)

In [2]:
orig_dir = os.getcwd()

# Lees predictie functies

In [3]:
# Zorg er voor dat de veranderingen in verbruik_predictie ook meegenomen worden
from imp import reload
import verbruik_predictie as vp
reload(vp)


<module 'verbruik_predictie' from 'C:\\Users\\MarcelKoolwijk\\Documents\\SQL\\jads_enexis\\notebooks\\verbruik_predictie.py'>

# Lees kleinverbruik data

In [4]:
# variables used in script
data_processed_location = '../data/processed'

if 'processed' not in os.getcwd():
    os.chdir(data_processed_location)

In [5]:
# kleinverbruikgegevens gegevens inlezen
df = pd.read_hdf('kleinverbruikgegevens_data.h5')

#Delete 2021 data by keeping JAAR < 2021
df = df[df['JAAR'] < 2021]

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


# Voeg Features toe
Voeg profiel toe op basis van SOORT_AANSLUITING en de SJV_TOTAAL per profiel

In [6]:
# Spoor 2. Voeg profiel toe aan verbruiksgegevens. SOORT_AANSLUITING bepaald het profiel
# Omdat de E2 profielen in de NEDU set niet consistent zijn (verschil tussen 2010-2017 en 2018-), gebruiken we deze niet
profiel_E1 = ['1X25','3X25', '1X20', '1x25', '3x25', '1x20'] # de rest is profiel E2

# Voeg in de kleinverbruikgegevens, het bijpassende profiel voor de soort aansluiting toe
def vervang_door_profiel(aansluiting, postcode, percentage):
    # Bepaal basisprofiel
    profiel = 'E1' # if aansluiting in profiel_E1 else 'E2' # Gebruiken als we wel een goed E2 profiel hebben

    # Een laag tarief percentage lager dan 50% zetten we in een A profiel
    lh_profiel = 'A' if percentage < 50 else 'B'

    # Bepaal welk laag tarief gebied de postcode zit
    if profiel == 'E1' and lh_profiel == 'B':
        postcode_area = int(postcode[:2]) # PC4
        # Postcodes < 65 is Noord-Brabant of Limburg
        if postcode_area < 65:
            lh_profiel = 'C'
    return profiel + lh_profiel

df["PROFIEL"] = np.vectorize(vervang_door_profiel)(df.SOORT_AANSLUITING, df.POSTCODE_VAN, df.SJV_LAAG_TARIEF_PERC)
df.PC4 = df.PC4.astype('int')
print (f'#E1A = {df[df.PROFIEL == "E1A"].PROFIEL.count()}')
print (f'#E1B = {df[df.PROFIEL == "E1B"].PROFIEL.count()}')
print (f'#E1C = {df[df.PROFIEL == "E1C"].PROFIEL.count()}')

#E1A = 956271
#E1B = 52567
#E1C = 257304


In [7]:
# Voeg features toe per PC6
df["E1A_TOTAAL"] = df[df.PROFIEL == 'E1A'].SJV_TOTAAL
df["E1B_TOTAAL"] = df[df.PROFIEL == 'E1B'].SJV_TOTAAL
df["E1C_TOTAAL"] = df[df.PROFIEL == 'E1C'].SJV_TOTAAL
df["WEIGHTED_LEVERINGSRICHTING_PERC"] = df.AANSLUITINGEN_AANTAL * df.LEVERINGSRICHTING_PERC

# Rol op tot PC4

In [8]:
# Rol op tot PC4
df_verbruik = df.groupby(['PC4','JAAR']).agg({'SJV_TOTAAL':'sum', 'E1A_TOTAAL' : 'sum', 'E1B_TOTAAL' : 'sum', 'E1C_TOTAAL': 'sum', 'AANSLUITINGEN_AANTAL':'sum', 'WEIGHTED_LEVERINGSRICHTING_PERC': 'sum'})
df_verbruik['LEVERINGSRICHTING_PERC'] = df_verbruik['WEIGHTED_LEVERINGSRICHTING_PERC'] / df_verbruik['AANSLUITINGEN_AANTAL']
df_verbruik = df_verbruik.drop(columns=['WEIGHTED_LEVERINGSRICHTING_PERC'])
df_verbruik

SJV_TOTAAL  E1A_TOTAAL  E1B_TOTAAL  E1C_TOTAAL  \
PC4  JAAR                                                   
4251 2010     21802.0     17114.0         0.0      4688.0   
     2011     23084.0     18179.0         0.0      4905.0   
     2012     24006.0     18172.0         0.0      5834.0   
     2013     23728.0     18421.0         0.0      5307.0   
     2014     23623.0     22610.0         0.0      1013.0   
...               ...         ...         ...         ...   
9999 2016        56.0        56.0         0.0         0.0   
     2017        53.0        53.0         0.0         0.0   
     2018       174.0       174.0         0.0         0.0   
     2019       163.0       163.0         0.0         0.0   
     2020       155.0       155.0         0.0         0.0   

           AANSLUITINGEN_AANTAL  LEVERINGSRICHTING_PERC  
PC4  JAAR                                                
4251 2010                  4325               99.949133  
     2011                  4413               99.950147  
     2012                  4486               99.848417  
     2013                  4512               99.578901  
     2014                  4544               98.932218  
...                         ...                     ...  
9999 2016                    21               81.000000  
     2017                    21               71.000000  
     2018                    37               64.865676  
     2019                    37               62.164054  
     2020                    37               54.052432  

[17130 rows x 6 columns]

In [9]:
# Verhuis de multi-level index naar kolommen en maak de index een simpele range van getallen
df_verbruik['PC4'] = df_verbruik.index.get_level_values('PC4')
df_verbruik['JAAR'] = df_verbruik.index.get_level_values('JAAR')
df_verbruik.index = range(len(df_verbruik))
df_verbruik.SJV_TOTAAL = df_verbruik.SJV_TOTAAL.astype('int')
df_verbruik.E1A_TOTAAL = df_verbruik.E1A_TOTAAL.astype('int')
df_verbruik.E1B_TOTAAL = df_verbruik.E1B_TOTAAL.astype('int')
df_verbruik.E1C_TOTAAL = df_verbruik.E1C_TOTAAL.astype('int')
df_verbruik

,SJV_TOTAAL,E1A_TOTAAL,E1B_TOTAAL,E1C_TOTAAL,AANSLUITINGEN_AANTAL,LEVERINGSRICHTING_PERC,PC4,JAAR
0,21802,17114,0,4688,4325,99.949133,4251,2010
1,23084,18179,0,4905,4413,99.950147,4251,2011
2,24006,18172,0,5834,4486,99.848417,4251,2012
3,23728,18421,0,5307,4512,99.578901,4251,2013
4,23623,22610,0,1013,4544,98.932218,4251,2014
...,...,...,...,...,...,...,...,...
17125,56,56,0,0,21,81.000000,9999,2016
17126,53,53,0,0,21,71.000000,9999,2017
17127,174,174,0,0,37,64.865676,9999,2018
17128,163,163,0,0,37,62.164054,9999,2019


# Voeg predictie toe voor de jaren 2021 tot 2023

In [10]:
# Voorspel het verbruik met lineaire regressie
df_pred = vp.predict_verbruik_lr(df_verbruik, predict_type='mid')
df_pred[df_pred.PC4 == 4251]

100%|█████████████████████████████████████████████████████████████████████████████| 1645/1645 [00:10<00:00, 160.10it/s]


,SJV_TOTAAL,E1A_TOTAAL,E1B_TOTAAL,E1C_TOTAAL,AANSLUITINGEN_AANTAL,LEVERINGSRICHTING_PERC,PC4,JAAR
0,22442.327273,20900.509091,0.0,1541.218182,4797.254545,92.771728,4251,2021
1,22369.336364,21107.381818,0.0,1261.254545,4835.509091,92.011754,4251,2022
2,22296.345455,21314.254545,0.0,981.290909,4873.763636,91.251780,4251,2023


In [11]:
accuracy = vp.calc_accuracy_lr(df_verbruik)
accuracy

{'r2_train': 8.410242799850032e-06,
 'r2_test': -0.003664233452043586,
 'MSE_train': 54704272.756639056,
 'MSE test': 50657203.6206131,
 'RMSE train': 7396.233687265368,
 'RMSE test': 7117.387415380246,
 'MAE train': 5858.4002013110785,
 'MAE test': 5728.186721532426,
 'MAPE train': 503.7932697301765,
 'MAPE test': 599.2498281955808}

In [12]:
# Check de merge voor postcode 4251
df_verbruik_en_pred = pd.merge(df_verbruik, df_pred, how='outer')
df_verbruik_en_pred[df_verbruik_en_pred.PC4 == 4251].head(14)

,SJV_TOTAAL,E1A_TOTAAL,E1B_TOTAAL,E1C_TOTAAL,AANSLUITINGEN_AANTAL,LEVERINGSRICHTING_PERC,PC4,JAAR
0,21802.000000,17114.000000,0.0,4688.000000,4325.000000,99.949133,4251,2010
1,23084.000000,18179.000000,0.0,4905.000000,4413.000000,99.950147,4251,2011
2,24006.000000,18172.000000,0.0,5834.000000,4486.000000,99.848417,4251,2012
3,23728.000000,18421.000000,0.0,5307.000000,4512.000000,99.578901,4251,2013
4,23623.000000,22610.000000,0.0,1013.000000,4544.000000,98.932218,4251,2014
5,22848.000000,21964.000000,0.0,884.000000,4570.000000,98.370897,4251,2015
6,22645.000000,21200.000000,0.0,1445.000000,4581.000000,97.923161,4251,2016
7,22742.000000,19830.000000,0.0,2912.000000,4661.000000,97.329758,4251,2017
8,22680.000000,20056.000000,0.0,2624.000000,4713.000000,95.501944,4251,2018
9,22391.000000,19537.000000,0.0,2854.000000,4720.000000,92.817822,4251,2019


# NEDU profielen

## Spoor 1
1. Lees NEDU profielen bestand in (nedu_files.h5)
2. Verwijder alle niet gebruikte profielen (E3, E4) en jaar 2021
3. Rol de kwartierdata op tot data per dag (optellen alle profielfactoren per kwartier)
4. Transformeer het dataframe om samenvoegen met verbruikdata mogelijk te maken op profiel:
    - Oud: (datum, E1A, E1B, E1C, E2A, E2B), bijv.:
        2010-01-01  0.003448  0.003514  0.003620  0.002332  0.003061
    - Nieuw: (datum, PROFIEL, FACTOR), bijv.:   
        2010-01-01  E1A  0.003448  
        2010-01-01  E1B  0.003514  
        2010-01-01  E1C  0.003620  
        2010-01-01  E2A  0.002332  
        2010-01-01  E2B  0.003061  
5. Vermenigvuldig de FACTOR kolom met 1e5 (stabiliteit model wordt dan beter)
5. Verdeel in train en test set op split datum '2018-01-01'
6. Creëer model en roep de fit functie aan op de train set
7. Bepaal de voorspelling op de test set en bereken de nauwkeurigheid (MAPE, RME, R2, )
8. Bepaal de voorspelling voor 2021-2023
9. Visualiseer de voorspellingen in 1 plot met meerdere kleuren mbv altair

In [13]:
# Lees NEDU profielen bestand in (nedu_files.h5)
# variables used in script
data_processed_location = '../data/processed'

if 'processed' not in os.getcwd():
    os.chdir(data_processed_location)

# NEDU profielen
df_nedu_profielen = pd.read_hdf('nedu_files.h5')
df_nedu_profielen_origineel = df_nedu_profielen

# mapping van PC4 buurt naar RES regio
df_pc4_res = pd.read_hdf('pc4_res.h5')

In [14]:
# Verwijder alle niet gebruikte profielen (E3, E4) 
drop_onnodige_profielen = {"E3A","E3B", 'E3C', 'E3D', 'E4A'}
df_nedu_profielen = df_nedu_profielen.drop(columns = drop_onnodige_profielen)
# Verwijder jaar 2021
df_nedu_profielen = df_nedu_profielen[df_nedu_profielen.DatumTijd < '2021-01-01']

In [15]:
# Rol de kwartierdata op tot data per dag (optellen alle profielfactoren per kwartier)
# feautures toevoegen zodat functie groupby werkt: jaar, maand, dag
df_nedu_profielen.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 385727 entries, 0 to 385726
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   DatumTijd  385727 non-null  datetime64[ns]
 1   E1A        385727 non-null  float64       
 2   E1B        385727 non-null  float64       
 3   E1C        385727 non-null  float64       
 4   E2A        385727 non-null  float64       
 5   E2B        385727 non-null  float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 20.6 MB


In [16]:
df_nedu_profielen['jaar'] = df_nedu_profielen.DatumTijd.dt.year
df_nedu_profielen['maand'] = df_nedu_profielen.DatumTijd.dt.month
df_nedu_profielen['dag'] = df_nedu_profielen.DatumTijd.dt.day

df_nedu_profielen = df_nedu_profielen.groupby(['jaar','maand','dag']).agg({'E1A':'sum', 'E1B':'sum', 'E1C':'sum', 'E2A':'sum', 'E2B':'sum'})

def maak_datum(jaar,maand,dag):
    return format(jaar,'04d') + '-' + format(maand,'02d') + '-' + format(dag,'02d')
df_nedu_profielen.index = np.vectorize(maak_datum)(df_nedu_profielen.index.get_level_values('jaar'), df_nedu_profielen.index.get_level_values('maand'), df_nedu_profielen.index.get_level_values('dag'))

In [17]:
# Verschillende dataframes maken voor de verschillende profielen
# op basis van df_nedu_profielen dataframe
df_nedu_e1a = pd.DataFrame(df_nedu_profielen['E1A']).reset_index().rename(columns={"index":"DATUM","E1A": "VERBRUIKS_FACTOR"})
df_nedu_e1b = pd.DataFrame(df_nedu_profielen['E1B']).reset_index().rename(columns={"index":"DATUM","E1B": "VERBRUIKS_FACTOR"})
df_nedu_e1c = pd.DataFrame(df_nedu_profielen['E1C']).reset_index().rename(columns={"index":"DATUM","E1C": "VERBRUIKS_FACTOR"})
df_nedu_e2a = pd.DataFrame(df_nedu_profielen['E2A']).reset_index().rename(columns={"index":"DATUM","E2A": "VERBRUIKS_FACTOR"})
df_nedu_e2b = pd.DataFrame(df_nedu_profielen['E2B']).reset_index().rename(columns={"index":"DATUM","E2B": "VERBRUIKS_FACTOR"})

# Vermenigvuldig de FACTOR kolom met 1e5 (stabiliteit model wordt dan beter)
df_nedu_e1a['VERBRUIKS_FACTOR'] = df_nedu_e1a['VERBRUIKS_FACTOR'] * 1e5
df_nedu_e1b['VERBRUIKS_FACTOR'] = df_nedu_e1b['VERBRUIKS_FACTOR'] * 1e5
df_nedu_e1c['VERBRUIKS_FACTOR'] = df_nedu_e1c['VERBRUIKS_FACTOR'] * 1e5
df_nedu_e2a['VERBRUIKS_FACTOR'] = df_nedu_e2a['VERBRUIKS_FACTOR'] * 1e5
df_nedu_e2b['VERBRUIKS_FACTOR'] = df_nedu_e2b['VERBRUIKS_FACTOR'] * 1e5

In [18]:
# Prophet
# train model per profiel voor bepaalde train periode
from_date = '2010-01-01'
split_date = '2019-01-01'

nedu.train_model_nedu_profielen_prophet(df_nedu_e1a, profile='e1a', from_date=from_date, split_date=split_date)
nedu.train_model_nedu_profielen_prophet(df_nedu_e1b, profile='e1b', from_date=from_date, split_date=split_date)
nedu.train_model_nedu_profielen_prophet(df_nedu_e1c, profile='e1c', from_date=from_date, split_date=split_date)
nedu.train_model_nedu_profielen_prophet(df_nedu_e2a, profile='e2a', from_date=from_date, split_date=split_date)
nedu.train_model_nedu_profielen_prophet(df_nedu_e2b, profile='e2b', from_date=from_date, split_date=split_date)

Length full     dataset: 4018 samples
Length training dataset: 3288 samples
Length test     dataset: 730 samples

Measures for Profile e1a:
R^2   test: 0.94
MSE   test: 88.93
RMSE  test: 9.43
MAE   test: 7.20
MAPE  test: 2.59
MAPA  test: 97.41%

Model saved: NEDU_PRO_e1a_2019-01-01.pkl

Length full     dataset: 4018 samples
Length training dataset: 3288 samples
Length test     dataset: 730 samples

Measures for Profile e1b:
R^2   test: 0.89
MSE   test: 275.85
RMSE  test: 16.61
MAE   test: 14.31
MAPE  test: 5.30
MAPA  test: 94.70%

Model saved: NEDU_PRO_e1b_2019-01-01.pkl

Length full     dataset: 4018 samples
Length training dataset: 3288 samples
Length test     dataset: 730 samples

Measures for Profile e1c:
R^2   test: 0.91
MSE   test: 225.25
RMSE  test: 15.01
MAE   test: 12.87
MAPE  test: 4.78
MAPA  test: 95.22%

Model saved: NEDU_PRO_e1c_2019-01-01.pkl

Length full     dataset: 4018 samples
Length training dataset: 3288 samples
Length test     dataset: 730 samples

Measures for Pro

In [19]:
# maak een LOW prediction voor een profiel voor een bepaalde periode
predict_start = '2021-01-01'
predict_end   = '2023-12-31'
predict_type  = 'low'
df_nedu_e1a_low_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e1a_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e1b_low_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e1b_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e1c_low_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e1c_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e2a_low_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e2a_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e2b_low_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e2b_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)

In [20]:
# maak een MID prediction voor een profiel voor een bepaalde periode
predict_start = '2021-01-01'
predict_end   = '2023-12-31'
predict_type  = 'mid'
df_nedu_e1a_mid_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e1a_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e1b_mid_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e1b_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e1c_mid_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e1c_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e2a_mid_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e2a_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e2b_mid_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e2b_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)

In [21]:
# maak een HIGH prediction voor een profiel voor een bepaalde periode
predict_start = '2021-01-01'
predict_end   = '2023-12-31'
predict_type  = 'high'
df_nedu_e1a_high_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e1a_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e1b_high_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e1b_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e1c_high_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e1c_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e2a_high_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e2a_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e2b_high_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e2b_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)

In [22]:
# DATUM aanpassen naar datetime formaat
df_nedu_e1a['DATUM'] = pd.to_datetime(df_nedu_e1a['DATUM'])
df_nedu_e1b['DATUM'] = pd.to_datetime(df_nedu_e1b['DATUM'])
df_nedu_e1c['DATUM'] = pd.to_datetime(df_nedu_e1c['DATUM'])
df_nedu_e2a['DATUM'] = pd.to_datetime(df_nedu_e2a['DATUM'])
df_nedu_e2b['DATUM'] = pd.to_datetime(df_nedu_e2b['DATUM'])

In [23]:
# Interactive zoom on both X and Y axis with two lower graphs.
df_nedu = df_nedu_e1a
df_nedu_pred_low = df_nedu_e1a_low_pred
df_nedu_pred_mid = df_nedu_e1a_mid_pred
df_nedu_pred_high = df_nedu_e1a_high_pred

interval = alt.selection_interval(encodings=['x'])
verbruik_range = alt.selection_interval(encodings=['y'])

base = alt.Chart(df_nedu.reset_index()).mark_line(clip=True).encode(
    x='DATUM:T',
    y='VERBRUIKS_FACTOR:Q'
)

forecast_low = alt.Chart(df_nedu_pred_low.reset_index()).mark_line(clip=True).encode(
                     x='DATUM:T',
                     y='VERBRUIKS_FACTOR:Q'
                     )
    
forecast_mid = alt.Chart(df_nedu_pred_mid.reset_index()).mark_line(clip=True).encode(
                     x='DATUM:T',
                     y='VERBRUIKS_FACTOR:Q'
                     )

forecast_high = alt.Chart(df_nedu_pred_high.reset_index()).mark_line(clip=True).encode(
                     x='DATUM:T',
                     y='VERBRUIKS_FACTOR:Q'
                     )
    
chart = base.encode(
    x=alt.X('DATUM:T', scale=alt.Scale(domain=interval.ref())),
    y=alt.Y('VERBRUIKS_FACTOR:Q', scale=alt.Scale(domain=verbruik_range.ref()))
).properties(
    width=800,
    height=400)

forecast_low_chart = forecast_low.encode(
    x=alt.X('DATUM:T', scale=alt.Scale(domain=interval.ref())),
    y=alt.Y('VERBRUIKS_FACTOR:Q', scale=alt.Scale(domain=verbruik_range.ref())),color=alt.value('orange')
).properties(
    width=800,
    height=400)

forecast_mid_chart = forecast_mid.encode(
    x=alt.X('DATUM:T', scale=alt.Scale(domain=interval.ref())),
    y=alt.Y('VERBRUIKS_FACTOR:Q', scale=alt.Scale(domain=verbruik_range.ref())),color=alt.value('red')
).properties(
    width=800,
    height=400)

forecast_high_chart = forecast_high.encode(
    x=alt.X('DATUM:T', scale=alt.Scale(domain=interval.ref())),
    y=alt.Y('VERBRUIKS_FACTOR:Q', scale=alt.Scale(domain=verbruik_range.ref())),color=alt.value('orange')
).properties(
    width=800,
    height=400)
    
view = base+forecast_low.add_selection(
    interval
).properties(
    width=800,
    height=50,
)

window = base+forecast_low.add_selection(
    verbruik_range
).properties(
    width=800,
    height=100,
)

chart + forecast_low_chart + forecast_mid_chart + forecast_high_chart & view & window

alt.VConcatChart(...)

In [24]:
# MID prediction dataframes samenvoegen met bestaande NEDU dataframes
df_nedu_e1a_all = pd.concat([df_nedu_e1a[df_nedu_e1a['DATUM'] < predict_start], df_nedu_e1a_mid_pred])
df_nedu_e1b_all = pd.concat([df_nedu_e1b[df_nedu_e1b['DATUM'] < predict_start], df_nedu_e1b_mid_pred])
df_nedu_e1c_all = pd.concat([df_nedu_e1c[df_nedu_e1c['DATUM'] < predict_start], df_nedu_e1c_mid_pred])
df_nedu_e2a_all = pd.concat([df_nedu_e2a[df_nedu_e2a['DATUM'] < predict_start], df_nedu_e2a_mid_pred])
df_nedu_e2b_all = pd.concat([df_nedu_e2b[df_nedu_e2b['DATUM'] < predict_start], df_nedu_e2b_mid_pred])

In [25]:
# verbruiksfactor weer delen door 1e5
df_nedu_e1a_all['VERBRUIKS_FACTOR'] = df_nedu_e1a_all['VERBRUIKS_FACTOR'] / 1e5
df_nedu_e1b_all['VERBRUIKS_FACTOR'] = df_nedu_e1b_all['VERBRUIKS_FACTOR'] / 1e5
df_nedu_e1c_all['VERBRUIKS_FACTOR'] = df_nedu_e1c_all['VERBRUIKS_FACTOR'] / 1e5
df_nedu_e2a_all['VERBRUIKS_FACTOR'] = df_nedu_e2a_all['VERBRUIKS_FACTOR'] / 1e5
df_nedu_e2b_all['VERBRUIKS_FACTOR'] = df_nedu_e2b_all['VERBRUIKS_FACTOR'] / 1e5

In [26]:
df_nedu_e1a_all2 = df_nedu_e1a_all.rename(columns={'VERBRUIKS_FACTOR': 'E1A'})
df_nedu_e1b_all2 = df_nedu_e1b_all.rename(columns={'VERBRUIKS_FACTOR': 'E1B'})
df_nedu_e1c_all2 = df_nedu_e1c_all.rename(columns={'VERBRUIKS_FACTOR': 'E1C'})
df_nedu_e2a_all2 = df_nedu_e2a_all.rename(columns={'VERBRUIKS_FACTOR': 'E2A'})
df_nedu_e2b_all2 = df_nedu_e2b_all.rename(columns={'VERBRUIKS_FACTOR': 'E2B'})

In [27]:
# de dataframes samenvoegen
df_nedu_profielen = pd.merge(df_nedu_e1a_all2, df_nedu_e1b_all2, on='DATUM', how='left')
df_nedu_profielen = pd.merge(df_nedu_profielen, df_nedu_e1c_all2, on='DATUM', how='left')
df_nedu_profielen = pd.merge(df_nedu_profielen, df_nedu_e2a_all2, on='DATUM', how='left')
df_nedu_profielen = pd.merge(df_nedu_profielen, df_nedu_e2b_all2, on='DATUM', how='left')

In [28]:
# jaar toevoegen aan nedu profielen om op te kunnen joinen met jaar verbruiksdata
df_nedu_profielen['JAAR'] = df_nedu_profielen['DATUM'].dt.year

In [29]:
df_nedu_profielen[df_nedu_profielen['DATUM'] >= '2021-01-01'].head()

,DATUM,E1A,E1B,E1C,E2A,E2B,JAAR
4018,2021-01-01,0.003325,0.003369,0.003395,0.002456,0.002604,2021
4019,2021-01-02,0.003415,0.003476,0.003551,0.002491,0.002887,2021
4020,2021-01-03,0.003411,0.003419,0.003516,0.002051,0.002507,2021
4021,2021-01-04,0.003401,0.003359,0.003331,0.003319,0.003151,2021
4022,2021-01-05,0.003274,0.003302,0.003274,0.003436,0.003206,2021


# Combinatiespoor
Combineer de dataframes van de NEDU profielen met voorspelling en het dataframe van de kleinverbruikgegevens inclusief voorspelling

In [30]:
# Combineer NEDU profielen met de kleinverbruikgegevens
df_combined = pd.merge(df_nedu_profielen, df_verbruik_en_pred, on=['JAAR'], how='left')

In [31]:
df_combined[df_combined.PC4 == 4251].tail(14)

,DATUM,E1A,E1B,E1C,E2A,E2B,JAAR,SJV_TOTAAL,E1A_TOTAAL,E1B_TOTAAL,E1C_TOTAAL,AANSLUITINGEN_AANTAL,LEVERINGSRICHTING_PERC,PC4
7875351,2023-12-18,0.003374,0.003375,0.003350,0.003435,0.003121,2023,22296.345455,21314.254545,0.0,981.290909,4873.763636,91.25178,4251
7876848,2023-12-19,0.003269,0.003339,0.003321,0.003518,0.003169,2023,22296.345455,21314.254545,0.0,981.290909,4873.763636,91.25178,4251
7878345,2023-12-20,0.003311,0.003383,0.003387,0.003428,0.003181,2023,22296.345455,21314.254545,0.0,981.290909,4873.763636,91.25178,4251
7879842,2023-12-21,0.003308,0.003380,0.003386,0.003483,0.003197,2023,22296.345455,21314.254545,0.0,981.290909,4873.763636,91.25178,4251
7881339,2023-12-22,0.003354,0.003397,0.003396,0.003231,0.003173,2023,22296.345455,21314.254545,0.0,981.290909,4873.763636,91.25178,4251
7882836,2023-12-23,0.003422,0.003511,0.003596,0.002542,0.002861,2023,22296.345455,21314.254545,0.0,981.290909,4873.763636,91.25178,4251
7884333,2023-12-24,0.003431,0.003466,0.003578,0.002074,0.002476,2023,22296.345455,21314.254545,0.0,981.290909,4873.763636,91.25178,4251
7885830,2023-12-25,0.003502,0.003490,0.003553,0.002545,0.002684,2023,22296.345455,21314.254545,0.0,981.290909,4873.763636,91.25178,4251
7887327,2023-12-26,0.003383,0.003444,0.003515,0.002506,0.002664,2023,22296.345455,21314.254545,0.0,981.290909,4873.763636,91.25178,4251
7888824,2023-12-27,0.003353,0.003404,0.003419,0.003321,0.003180,2023,22296.345455,21314.254545,0.0,981.290909,4873.763636,91.25178,4251


In [32]:
# Voeg nieuwe kolom VERBRUIK toe die de som is van het verbruik van de verschillende profielen
df_combined['VERBRUIK'] = df_combined.E1A * df_combined.E1A_TOTAAL + df_combined.E1B * df_combined.E1B_TOTAAL + df_combined.E1C * df_combined.E1C_TOTAAL

# Gooi de tussenkolommen weg. Die hebben we niet meer nodig
df_combined = df_combined.drop(columns=['E1A','E1B','E1C','E1A_TOTAAL','E1B_TOTAAL','E1C_TOTAAL','JAAR'])

In [33]:
# cross check. Het totaal van een profiel over een heel jaar moet 1 zijn. Dat betekent dat het totaal van de verbruiken gelijk moet zijn aan SJV_TOTAAL
print (f"Totaal verbruik = {df_combined[(df_combined.DATUM < datetime(2011,1,1)) & (df_combined.PC4 == 4251)].agg({'VERBRUIK':'sum'}).values[0]}")
print (f"SJV totaal = {df_combined[(df_combined.DATUM < datetime(2011,1,1)) & (df_combined.PC4 == 4251)].SJV_TOTAAL.values[0]}")

Totaal verbruik = 21800.70682874
SJV totaal = 21802.0


In [37]:
df_combined[df_combined.PC4 == 4251].tail(14)

,DATUM,E2A,E2B,SJV_TOTAAL,AANSLUITINGEN_AANTAL,LEVERINGSRICHTING_PERC,PC4,VERBRUIK
7875351,2023-12-18,0.003435,0.003121,22296.345455,4873.763636,91.25178,4251,75.211485
7876848,2023-12-19,0.003518,0.003169,22296.345455,4873.763636,91.25178,4251,72.933621
7878345,2023-12-20,0.003428,0.003181,22296.345455,4873.763636,91.25178,4251,73.892844
7879842,2023-12-21,0.003483,0.003197,22296.345455,4873.763636,91.25178,4251,73.821958
7881339,2023-12-22,0.003231,0.003173,22296.345455,4873.763636,91.25178,4251,74.811417
7882836,2023-12-23,0.002542,0.002861,22296.345455,4873.763636,91.25178,4251,76.458096
7884333,2023-12-24,0.002074,0.002476,22296.345455,4873.763636,91.25178,4251,76.644762
7885830,2023-12-25,0.002545,0.002684,22296.345455,4873.763636,91.25178,4251,78.130958
7887327,2023-12-26,0.002506,0.002664,22296.345455,4873.763636,91.25178,4251,75.546264
7888824,2023-12-27,0.003321,0.003180,22296.345455,4873.763636,91.25178,4251,74.822503
